In [1]:
import pandas as pd

# Loading and getting to know the dataset

In [2]:
df = pd.read_csv('adventure_works_purchase_orders.csv')

# df.info()
# df.head()

# Columns seems to bring no value, therefore dropping them
df[['ShipDate', 'DueDate']].drop_duplicates()
usable_cols = [col for col in df.columns if col not in ['ShipDate', 'DueDate']]
df = df[usable_cols]

# Removing underscore from column names so every header is standardized
col_format = [i.replace('_', '') for i in usable_cols]
new_cols = dict(zip(usable_cols, col_format))
df.rename(columns=new_cols, inplace=True)

# Converting OrderDate to date format
df['OrderDate'] = pd.to_datetime(df['OrderDate'])

# Data Wrangling

In [3]:
# Adding sales data
# Actual sales = Unit Price * Discount * Quantity
df['ActualSales'] = df['UnitPrice'] * (1-df['UnitPriceDiscount']) * df['OrderQty']
# Calculating COGS (Cost of Goods Sold) = Unit_Cost * Quantity
df['COGS'] = df['UnitCost'] * df['OrderQty']
# Front Marign are actual sales minus COGS
df['FrontMargin'] = df['ActualSales'] - df['COGS'] 
# Deducting shipping cost from FM to get Gross Margin
df['GrossMargin'] = df['FrontMargin'] - df['OrderQty'] * df['UnitFreightCost']
# Adding a helper column by which promo share is going to be calculated
df['IsPromo'] = 1
df.loc[df['UnitPriceDiscount'] == 0, "IsPromo"] = 0

In [4]:
# Creating set offseted by 1 year so historical changes can be calculated
df_y_offset = df.copy()
df_y_offset['OrderDate'] = df_y_offset['OrderDate'] + pd.DateOffset(years=1)

In [5]:
# Merging the dataset with offseted dataset so Like for Like can be calculated
sales_data = pd.merge(df, df_y_offset,
                      how='outer',
                      on=['OrderDate',
                          'SalesPerson',
                          'SalesRegion',
                          'SalesProvince',	
                          'SalesCity',
                          'SalesPostalCode',
                          'CustomerCode',
                          'CustomerName',
                          'CustomerRegion',
                          'CustomerProvince',
                          'CustomerCity',
                          'CustomerPostalCode',
                          'ProductCategory',
                          'ProductSubCategory',
                          'ProductName',
                          'ProductCode'],
                      suffixes=('_CY', '_LY'))


In [6]:
new_cols =['SalesOrderNumber_CY', 'OrderDate', 'SalesPerson', 'SalesRegion',
           'SalesProvince', 'SalesCity', 'SalesPostalCode', 'CustomerCode',
           'CustomerName', 'CustomerRegion', 'CustomerProvince',
           'CustomerCity', 'CustomerPostalCode',
           'ProductCategory', 'ProductSubCategory', 'ProductName',
           'ProductCode', 'UnitCost_CY', 'UnitPrice_CY', 'UnitPriceDiscount_CY',
           'OrderQty_CY', 'UnitFreightCost_CY', 'ActualSales_CY', 'COGS_CY',
           'FrontMargin_CY', 'GrossMargin_CY', 'IsPromo_CY', 'SalesOrderNumber_LY',
           'UnitCost_LY', 'UnitPrice_LY', 'UnitPriceDiscount_LY',
           'OrderQty_LY', 'UnitFreightCost_LY', 'ActualSales_LY', 'COGS_LY',
           'FrontMargin_LY', 'GrossMargin_LY', 'IsPromo_LY']

sales_data = sales_data[new_cols]

# Removing suffixes
discrete_cols =['OrderDate', 'SalesPerson', 'SalesRegion_CY',
                'SalesProvince_CY', 'SalesCity_CY', 'SalesPostalCode', 'CustomerCode',
                'CustomerName_CY', 'CustomerRegion_CY', 'CustomerProvince_CY',
                'CustomerCity_CY', 'CustomerPostalCode',
                'ProductCategory_CY', 'ProductSubCategory_CY', 'ProductName_CY',
                'ProductCode']
discrete_cols_new = []

for col in discrete_cols:
    discrete_cols_new.append(col.replace('_CY', ''))
    
discrete_cols_dict = dict(zip(discrete_cols, discrete_cols_new))
sales_data.rename(columns=discrete_cols_dict, inplace=True)

# Data Validation

In [7]:
# Summarazing ActualSales columns to check if data got lost during processing
original_summary = pd.DataFrame(df.groupby(df.OrderDate.dt.year)['ActualSales'].sum())
new_summary = pd.DataFrame(sales_data.groupby(sales_data.OrderDate.dt.year)[['ActualSales_CY', 'ActualSales_LY']].sum())
# Shifting offseted column back to it's original position for easier check
new_summary['ActualSales_LY'] = new_summary['ActualSales_LY'].shift(-1)

combined_summary = pd.merge(original_summary, new_summary, how='inner', on=['OrderDate'])
combined_summary['VS_Original'] = combined_summary['ActualSales'] - combined_summary['ActualSales_CY']
combined_summary['VS_Offset'] = combined_summary['ActualSales'] - combined_summary['ActualSales_LY']
combined_summary.head()

,ActualSales,ActualSales_CY,ActualSales_LY,VS_Original,VS_Offset
OrderDate,,,,,
2005,1.115334e+07,1.115334e+07,1.115334e+07,0.0,0.0
2006,3.275189e+07,3.275189e+07,3.275189e+07,0.0,0.0
2007,3.845821e+07,3.845821e+07,3.845821e+07,0.0,0.0
2008,1.776719e+07,1.776719e+07,1.776719e+07,0.0,0.0


In [8]:
sales_data.groupby(sales_data.OrderDate.dt.year)[['SalesOrderNumber_CY', 'SalesOrderNumber_LY']].nunique()

,SalesOrderNumber_CY,SalesOrderNumber_LY
OrderDate,,
2005,286,0
2006,764,286
2007,1020,764
2008,559,1020
2009,0,559


In [9]:
l4l_sales_data = sales_data[sales_data['ActualSales_CY'].notna() & sales_data['ActualSales_LY'].notna()]
l4l_sales_data.groupby(l4l_sales_data.OrderDate.dt.year)[['ActualSales_CY', 'ActualSales_LY']].sum()

,ActualSales_CY,ActualSales_LY
OrderDate,,
2006,1.327140e+06,9.133279e+05
2007,4.936375e+06,4.809734e+06
2008,5.373082e+06,5.449732e+06


# Data Export

In [10]:
# Discarding year 2009 as it is result of offseting the year, therefore non-existent
sales_data = sales_data[sales_data['OrderDate'].dt.year != 2009]
# Saving the new datasets
sales_data.to_csv('cake_techi_data.csv', index=False)
l4l_sales_data.to_csv('cake_techi_l4l_data.csv', index=False)